# The Battle of Neighborhood: Japanese Restaurant compare in USA

#### Data Science Capstone Final Project - IBM Data Science Professional Certificate on Coursera

### Table of Content:

1. Introduction (wk4)<br>
2. Data Source(wk4)<br>
3. Data Preparation/Pre-processing(wk5)<br>
4. Analysis(wk5)<br>
5. Conclusion(wk5)<br>


#### 1. Introduction (wk4)

Japanese cuisine always attracts people no matter is japanses or from other countries. As someone who travel or move from Janpan to the US, or someone are addicted to japanses cuisine living in the US, when they are planning for which city to stay in, the density of Japanese restaurant would be a key factor for them to consider. So 3 cities in the US are compared in this project to find out which one may be the best choice for people who are interested in Japanese food. The 3 cities are: New York City, Miami and Seattle. 

#### 2. Data Source (wk4)

In this project, FourSquare API is used to collect data about locations of Japanese Restaurants in 3 US cities.

#### 3. Data Preparation/Pre-processing(wk5)

import all the dependencies 

In [5]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests 
from pandas.io.json import json_normalize 
import folium 

print('Libraries imported.')

Libraries imported.


In [6]:
CLIENT_ID = 'M41YLUTSKS1ZT5RHPCGZVYLO5APQ0O2W5A3DEMQJKZ5W1Z0I'
CLIENT_SECRET = 'JELILA5WQEAWWUO0GELIX4GLIG3FKGEPXZHES1BYHAOPJ3BT'
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: M41YLUTSKS1ZT5RHPCGZVYLO5APQ0O2W5A3DEMQJKZ5W1Z0I


In [10]:
# type your answer here
LIMIT = 100 # Maximum is 100
cities = ["New York, NY", 'Seattle, WA', 'Miami, FL']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d111941735") # Japanese Restaurant CATEGORY ID
    results[city] = requests.get(url).json()

In [11]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [13]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Japanese Restaurant in {city} = ", results[city]['response']['totalResults'])
    print("Done")

Total number of Japanese Restaurant in New York, NY =  359
Done
Total number of Japanese Restaurant in Seattle, WA =  235
Done
Total number of Japanese Restaurant in Miami, FL =  196
Done


In [14]:
maps[cities[0]]

In [15]:
maps[cities[1]]

In [16]:
maps[cities[2]]